## **Data Gathering**

In [ ]:
pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np


query = "(from:elonmusk) until:2023-01-01 since:2005-01-01"
tweets = []
limit = 100000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content,tweet.likeCount, tweet.retweetCount])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','Like_Count','Retweet_Count'])
print(df)

# to save to csv
# df.to_csv('tweets.csv')

<ipython-input-26-a2ddae5574a8>:18: FutureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content,tweet.likeCount, tweet.retweetCount])


                           Date      User  \
0     2022-12-07 10:36:28+00:00  elonmusk   
1     2022-12-07 04:55:11+00:00  elonmusk   
2     2022-12-07 02:45:54+00:00  elonmusk   
3     2022-12-07 02:40:08+00:00  elonmusk   
4     2022-12-06 23:42:19+00:00  elonmusk   
...                         ...       ...   
18577 2011-12-03 08:22:07+00:00  elonmusk   
18578 2011-12-03 08:20:28+00:00  elonmusk   
18579 2011-12-01 10:29:04+00:00  elonmusk   
18580 2011-12-01 09:55:11+00:00  elonmusk   
18581 2010-06-04 18:31:57+00:00  elonmusk   

                                                   Tweet  Like_Count  \
0      Oh, you know, keeping busy … https://t.co/jgGY...       91468   
1      @ESYudkowsky To be called AGI, it needs to inv...       16691   
2                           @charliekirk11 Small world …      125464   
3      Oh, what a tangled web they weave, when first ...      275214   
4      @EricRWeinstein @bariweiss @mtaibbi Only disco...      118607   
...                        

## **Hashtags removal**

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# remove hashtags
def hashtags(text):
  hash = re.findall(r"#(\w+)", text)
  return hash

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!





## **Emojis translation**

In [ ]:
pip install emot


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
# translate emoji
def emoji(text):
  for emot in UNICODE_EMOJI:
    if text == None:
      text = text
    else:
      text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

## **Usernames removal**

In [ ]:
# remove retweet username and tweeted at @username
def remove_users(tweet):
  '''Takes a string and removes retweet and @user information'''
  tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
  # remove tweeted at
  return tweet

## **Links removal**

In [ ]:
# remove links
def remove_links(tweet):
  '''Takes a string and removes web links from it'''
  tweet = re.sub(r'http\S+', '', tweet) # remove http links
  tweet = re.sub(r'bit.ly/\S+', '', tweet) # remove bitly links
  # tweet = tweet.strip('[link]') # remove [links]
  return tweet
def clean_html(text):
  html = re.compile('<.*?>')#regex
  return html.sub(r'',text)

## **Non-ASCII characters removaL**

In [ ]:
# remove non ascii character
def non_ascii(s):
  return "".join(i for i in s if ord(i)<128)

def lower(text):
  return text.lower()
  # remove email address
 

## **Email address and punctuation removal**

In [ ]:
def email_address(text):
  email = re.compile(r'[\w\.-]+@[\w\.-]+')
  return email.sub(r'',text)

def punct(text):
  token=RegexpTokenizer(r'\w+')#regex
  text = token.tokenize(text)
  text= " ".join(text)
  return text

## **Stopwords removal**

In [ ]:
# remove stopwords
def removeStopWords(str):
#select english stopwords
  cachedStopWords = set(stopwords.words("english"))
#add custom words
  cachedStopWords.update(('and','I','A','http','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these','mailto','regards','ayanna','like','email'))
#remove stop words
  new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
  return new_str

## **Special characters removal**

In [ ]:
def remove_(tweet):
  tweet = re.sub('([_]+)', "", tweet)
  return tweet

## **Data Preprocessing**

In [ ]:
#apply all the functions above
df['hashtag'] = df.Tweet.apply(func = hashtags)
df['new_tweet'] = df.Tweet.apply(func = emoji)
df['new_tweet'] = df.new_tweet.apply(func = remove_users)
df['new_tweet'] = df.new_tweet.apply(func = clean_html)
df['new_tweet'] = df.new_tweet.apply(func = remove_links)
df['new_tweet'] = df.new_tweet.apply(func = non_ascii)
df['new_tweet'] = df.new_tweet.apply(func = lower)
df['new_tweet'] = df.new_tweet.apply(func = email_address)
df['new_tweet'] = df.new_tweet.apply(func = removeStopWords)
df['new_tweet'] = df.new_tweet.apply(func = clean_html)
df['new_tweet'] = df.new_tweet.apply(func = punct)
df['new_tweet'] = df.new_tweet.apply(func = remove_)

In [ ]:
df

,Date,User,Tweet,Like_Count,Retweet_Count,hashtag,new_tweet
0,2022-12-07 10:36:28+00:00,elonmusk,"Oh, you know, keeping busy … https://t.co/jgGY...",91468,6957,[],oh know keeping busy
1,2022-12-07 04:55:11+00:00,elonmusk,"@ESYudkowsky To be called AGI, it needs to inv...",16691,992,[],called agi needs invent amazing things discove...
2,2022-12-07 02:45:54+00:00,elonmusk,@charliekirk11 Small world …,125464,7010,[],small world
3,2022-12-07 02:40:08+00:00,elonmusk,"Oh, what a tangled web they weave, when first ...",275214,24194,[],oh tangled web weave first practice
4,2022-12-06 23:42:19+00:00,elonmusk,@EricRWeinstein @bariweiss @mtaibbi Only disco...,118607,10596,[],discovered sunday
...,...,...,...,...,...,...,...
18577,2011-12-03 08:22:07+00:00,elonmusk,That was a total non sequitur btw,142,15,[],total non sequitur btw
18578,2011-12-03 08:20:28+00:00,elonmusk,"Great Voltaire quote, arguably better than Twa...",85,33,[],great voltaire quote arguably better twain hea...
18579,2011-12-01 10:29:04+00:00,elonmusk,I made the volume on the Model S http://t.co/w...,78,12,[],made volume model go 11 need work miniature st...
18580,2011-12-01 09:55:11+00:00,elonmusk,Went to Iceland on Sat to ride bumper cars on ...,191,22,[],went iceland sat ride bumper cars ice no count...


## **Sentiment Classification**

In [ ]:
!pip install modelzoo-client[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install scipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']

In [ ]:
def get_sentiment_label(tweet):
 
  encoded_tweet = tokenizer(tweet, return_tensors='pt')
  # print(encoded_tweet)
  output = model(**encoded_tweet)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  for i in range(len(scores)):
    
      l = labels[i]
      s = scores[i]
      # print(l,s)
  major_sentiment = labels[np.argmax(scores)] 
  return major_sentiment

In [ ]:
mask = df['Tweet'].str.contains("dogecoin",case=False)
df = df[mask]
df

,Date,User,Tweet,Like_Count,Retweet_Count,hashtag,new_tweet
1308,2022-10-17 05:52:27+00:00,elonmusk,@DogecoinNorway @moxie @ID_AA_Carmack 🤣 that’s...,1657,65,[],thats happened times
1951,2022-08-18 14:57:11+00:00,elonmusk,@DogecoinNorway @micsolana 🤣,2209,93,[],
2193,2022-07-21 03:46:18+00:00,elonmusk,@dogecoin Upgrades!,23913,1968,[],upgrades
2411,2022-06-19 06:19:16+00:00,elonmusk,I will keep supporting Dogecoin,384935,35564,[],keep supporting dogecoin
2507,2022-06-15 07:40:02+00:00,elonmusk,@DogecoinNorway @teslaownersSV @TheTexanNews @...,2169,103,[],
2732,2022-05-31 18:09:33+00:00,elonmusk,@BillyM2k @PinballReed @crikey_news @ummjackso...,11743,1194,[],youre humble bro billys sense humor amp irreve...
2734,2022-05-31 17:56:10+00:00,elonmusk,@PinballReed @crikey_news @ummjackson @cameron...,3543,307,[],palmer always forgets mention never wrote sing...
2737,2022-05-31 14:23:18+00:00,elonmusk,@DogecoinNorway @Louisbotelho17 @teslaownersSV...,2601,128,[],yeah
2876,2022-05-26 23:11:31+00:00,elonmusk,"@davidgokhshtein Dogecoin Trillionaire, the Mo...",16730,2166,[],dogecoin trillionaire movie
3226,2022-05-03 22:01:26+00:00,elonmusk,@DogecoinNorway @OpenSociety @georgesoros @mal...,2897,123,[],


In [ ]:
df['label'] = df['new_tweet'].apply(get_sentiment_label)

<ipython-input-44-1818e7705683>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['new_tweet'].apply(get_sentiment_label)


In [ ]:
df.to_csv('tweets.csv')
from google.colab import files
files.download('tweets.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>